# **<center>SETI Break through<center>**

First of all we have to deal with the required packages.  

In [78]:
import os

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC


Now we will read our labels and get the data it self.

In [148]:
train_dir = '/kaggle/input/seti-breakthrough-listen/train'
test_dir = '/kaggle/input/seti-breakthrough-listen/test'
train_df = pd.read_csv('/kaggle/input/seti-breakthrough-listen/train_labels.csv',nrows=10000)
test_df = pd.read_csv('/kaggle/input/seti-breakthrough-listen/sample_submission.csv',nrows=10000)

In [149]:
def return_filepath(name, folder=train_dir):
    path = os.path.join(folder, name[0], f'{name}.npy')
    return path

In [150]:
train_df['image_path'] = train_df['id'].apply(lambda x: return_filepath(x))
test_df['image_path'] = test_df['id'].apply(lambda x: return_filepath(x, folder=test_dir))
train_df.head()

,id,target,image_path
0,0024012d1431fbc,1,./train\0\0024012d1431fbc.npy
1,1004c26b9a3f877,1,./train\1\1004c26b9a3f877.npy
2,0000799a2b2c42d,0,./train\0\0000799a2b2c42d.npy
3,0012ef703e41ef0,0,./train\0\0012ef703e41ef0.npy
4,00042890562ff68,0,./train\0\00042890562ff68.npy
5,0028a35de92941d,1,./train\0\0028a35de92941d.npy


In [151]:
train_image_paths, valid_image_paths, train_targets, valid_targets = train_test_split(train_df['image_path'], train_df['target'], stratify=train_df['target'], random_state=42)

In [159]:
train_data = []
for train_image_path in train_image_paths:
    image = (np.load(train_image_paths[0]))
    
    # 1
  #  on = np.vstack((image[0],image[2],image[4]))
  #  off = np.vstack((image[1], image[3], image[5]))
  #  image_merged = np.concatenate((on, off), axis=1).transpose((1, 0))


    # 2
    image_merged = np.vstack(image).transpose((1, 0))

    # 3
  #  np.vstack((image[0],image[1],image[2],image[3],image[4],image[5]))

    # 4
  #  on = np.vstack((image[0],image[2],image[4]))
  #  off = np.vstack((image[1], image[3], image[5]))
  #  image_merged = np.concatenate((on, off), axis=1))

    train_data.append(image_merged.flatten())


0    0024012d1431fbc
4    00042890562ff68
Name: id, dtype: object

In [153]:
models = []
models.append(('LR', LogisticRegression(solver='liblinear', multi_class='ovr')))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC(gamma='auto')))


In [ ]:
results = []
names = []
for name, model in models:
    kfold = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)
    cv_results = cross_val_score(model, train_data, train_targets, cv=kfold, scoring='accuracy')
    results.append(cv_results)
    names.append(name)
    print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))
# Compare Algorithms
plt.boxplot(results, labels=names)
plt.title('Algorithm Comparison')
plt.show()